## Generating Data Table for Geocodes 

Attributes: 
- Geocode Level ({1: Country, 2: Region, 3: Coordinates})
- Location Name 
- ID (joining parameter) 
- Country
- geometery (either Multipolygon or Line)

*Additional attributes will be added to this table independently. This prepares the base data tables for use.*

This processing file should be used sparingly. Estimated time to run ~5 hours. All updates to the main files should be done independently of this notebook and documented in the [geocoding README](https://github.ncsu.edu/nakraft/LAS-BRI/blob/b73cc10b26fb458fbab79682da3fc2919395f519/data_final/geocode.md). 

*All areas where this process writes tables have been commented out to prevent overriding non-verified (or partial) data*

In [1]:
import pandas as pd 
import geopandas as gpd
import os 
import geocoder
import reverse_geocoder as rg
import json
import warnings 
warnings.filterwarnings('ignore')

# find this path; use releative locations for loading
git_repo_loc = os.path.dirname(os.path.realpath("Geocoding.ipynb"))

### Formulas to support Geocoding / Reverse Geocoding

In [2]:
# utilized to parse keys with unidentified names 
# dictionary: the dictionary to look through 
# substring: the string of interest within the values
def find_key(dictionary, substring):
    return dict(filter(lambda item: substring in item[0], dictionary.items()))

In [3]:
# all country data is stored in arcgis_countries 
# each country is assigned a unique id, this method returns that id given the country name as a string 
def get_country_id(country): 
    try:
        return arcgis_countries[arcgis_countries['country'] == country]['id'].values[0]
    except:
        # If there is difficulty locating it, a "Non" string will be returned. 
        # This will not restrict other processes, so pay attend to the print error alert
        print("This country " + str(country) + " was not found")
        return 'Non' 

In [4]:
# there is a dataframe here known as geobase_cities
# this method appends one city (and the associated data) to the dataframe 
# df: dataframe to append to 
# loc_name : as a string to append 
# idd: id of the city featured as xxxyy, where xxx is the countries 3 digit code 
# country: as a string 
# geometry: Point(x, y) coordinate 
# dataframe returned, caller method needs df = add_loc(...)
def add_loc(df, loc_name, idd, country, geometry): 
    row = {'name' : loc_name, 'id': idd, 'country': country, 'geometry': geometry}
    df = df.append(row, ignore_index=True)
    return df

In [5]:
# From a city name and country, identify the latitude and longitude coordinates 
# this processes uses Open Street Maps, which limits calls to 1/second  
# city_name: string of city to find 
# country: string of country that city remains in 
def geocode(city_name, country): 
    g = geocoder.osm(city_name + ", " + country)
    try: 
        lat = g.json['lat']
        long = g.json['lng']
    except: 
        # if no lat/long can be found
        return ("No", "Go")

    return (long, lat)

In [6]:
# AKA Reverse Geocoding - takes a point and finds the nearest city
# note: this can be somewhat innaccurate
# geoaid data custom shapes uses this method to create the city name - however they will also have the id of the 
# expenditure for reference 
# point: Point geometry as Point(x, y)
# return the string "Dallas, Texas" for location near the coordinates 
def find_nearby_city(point, level): 
    x = point.x
    y = point.y 
    g = rg.search((x, y))
    try: 
        if level == 'region': 
            name = g[0]['admin1']
        
        if (level == 'city') or (name == ''):
            name = g[0]['name']
        cc = g[0]['cc']
    except: 
        print(g)
        return "none identified"
    
    return name + ", " + cc

In [7]:
# this method is used to get the next ID for a city in the given country
# this DOES NOT check to see if that city already exists 
# country: string of country 
# loc_df: dataframe of cities/regions to check for the next id in
# returns the id (as a string) for the city record to be logged with 
def add_next_loc(country, loc_df): 
    # restrict to that country
    limit = loc_df[loc_df['country'] == country]
    # remove the first three places and sort the rest 
    limit['id'] = limit['id'].astype(str).str[3:].astype(int)
    limit.sort_values(by='id')
    
    try: 
        # if you cannot grab the last value, then this country has no associated data, start counting at 1
        last_id = limit['id'].iloc[-1]
    except: 
        last_id = 0
    
    # get the country id 
    country_id = str(get_country_id(country)).zfill(3)
    
    return str(country_id) + str(last_id + 1)

In [8]:
def load_dict(path): 
    file = open(path, "r")
    contents = file.read()
    dictionary = json.loads(contents)
    file.close()
    return dictionary

### Regional Level 1 (Countries) 

This is the main dataframe we will utilize for countries.

__Sources__
- ArcGIS 

*Note: there are boundary disputes. Entities have been stored in accordance with the closest proximity recognized area*

In [9]:
arcgis_countries_url = "https://opendata.arcgis.com/datasets/2b93b06dc0dc4e809d3c8db5cb96ba69_0.geojson"
arcgis_countries = gpd.read_file(arcgis_countries_url).drop(columns=['COUNTRYAFF', 'AFF_ISO', 'FID']).rename(columns={'COUNTRY': 'country', 'ISO': 'iso', 'SHAPE_Length': 'shape_length', 'SHAPE_Area' : 'shape_area'})
arcgis_countries['name'] = arcgis_countries['country']

#ensure naming conventions are consistent 
country_mapping = load_dict("../country_config.txt")

arcgis_countries['country'] = arcgis_countries['country'].replace(country_mapping)

In [10]:
# give the countries a standard naming convention 
# utilize 3 digits (include leading zeros as needed)
# identification is a code and has no relevance toward locality 

arcgis_countries['id'] = list(range(1, len(arcgis_countries) +1))
arcgis_countries['id'] = arcgis_countries['id'].apply(lambda x: str(x).zfill(3))

In [11]:
arcgis_countries = arcgis_countries[['name', 'id', 'country', 'geometry', 'iso', 'shape_length', 'shape_area']]
arcgis_countries

,name,id,country,geometry,iso,shape_length,shape_area
0,American Samoa,001,American Samoa,"MULTIPOLYGON (((-170.74390 -14.37555, -170.749...",AS,0.600124,0.013720
1,United States Minor Outlying Islands,002,United States Minor Outlying Islands,"MULTIPOLYGON (((-160.02114 -0.39805, -160.0281...",UM,0.480216,0.003216
2,Cook Islands,003,Cook Islands,"MULTIPOLYGON (((-159.74698 -21.25667, -159.793...",CK,0.980664,0.013073
3,French Polynesia,004,French Polynesia,"MULTIPOLYGON (((-149.17920 -17.87084, -149.258...",PF,3.930211,0.175332
4,Niue,005,Niue,"MULTIPOLYGON (((-169.89389 -19.14556, -169.930...",NU,0.541413,0.021414
...,...,...,...,...,...,...,...
244,Northern Mariana Islands,245,Northern Mariana Islands,"MULTIPOLYGON (((145.73468 15.08722, 145.72830 ...",MP,0.908853,0.019927
245,Palau,246,Palau,"MULTIPOLYGON (((134.53137 7.35444, 134.52234 7...",PW,1.105323,0.031136
246,Russian Federation,247,Russia,"MULTIPOLYGON (((-179.99999 68.98010, -179.9580...",RU,1536.287150,2931.526082
247,Spain,248,Spain,"MULTIPOLYGON (((-2.91472 35.27361, -2.93924 35...",ES,51.724956,52.915449


### Regional Level 3 (Coordinates) - part 1 - identify main sources

Provide a base database for city matches sourced from [simplemaps](https://simplemaps.com/data/world-cities)

*Ideally we would be calling to an API to grab any city; we can expand to this technique using [geoparsepy](https://pypi.org/project/geoparsepy/) or the [ArcGIS API](https://developers.arcgis.com/python/sample-notebooks/openstreetmap-exploration/)*

In [12]:
main_cities = pd.read_csv(git_repo_loc + "/worldcities.csv")

geo_cities = gpd.GeoDataFrame(
    main_cities, geometry=gpd.points_from_xy(main_cities.lng, main_cities.lat))
geo_cities = geo_cities.drop(columns=['city_ascii', 'lat', 'lng', 'iso3', 'capital', 'id', 'admin_name', 'iso2']).rename(columns={'city': 'name'})

In [13]:
geo_cities['country'] = geo_cities['country'].replace(country_mapping)

In [14]:
# create unique naming conventions based on locality
geo_cities = geo_cities.sort_values(by=['country', 'name']).reset_index()

# grab the country id 
mapping_country_codes = dict(zip(arcgis_countries.country, arcgis_countries.id))
geo_cities['country_id'] = geo_cities['country'].map(mapping_country_codes)

# add unique id within each country 
geo_cities['id']= geo_cities.groupby(['country_id']).cumcount()+1
geo_cities['id'] = geo_cities['id'].astype(str)

# append so that the country name has trailing zeros 
geo_cities['id'] = geo_cities['country_id'].str.cat(geo_cities['id'])

In [15]:
geo_cities = geo_cities[['name', 'id', 'country', 'geometry', 'population']]
geo_cities

,name,id,country,geometry,population
0,Asadābād,1921,Afghanistan,POINT (71.15280 34.87420),48400.0
1,Aībak,1922,Afghanistan,POINT (68.03940 36.25340),4938.0
2,Baghlān,1923,Afghanistan,POINT (68.70000 36.13280),73031.0
3,Balkh,1924,Afghanistan,POINT (66.89890 36.75810),77000.0
4,Barakī Barak,1925,Afghanistan,POINT (68.96670 33.96670),22305.0
...,...,...,...,...,...
40996,Redcliff,11522,Zimbabwe,POINT (29.78330 -19.03330),32346.0
40997,Ruwa,11523,Zimbabwe,POINT (31.24470 -17.88970),22038.0
40998,Shamva,11524,Zimbabwe,POINT (31.57000 -17.31960),10317.0
40999,Victoria Falls,11525,Zimbabwe,POINT (25.83330 -17.93330),33060.0


### Regional Level 2 (Regions) 

- Data is currently stored in a Git Repo 
- Shapefiles have been compiled externally and stored
- Non-geocoded data fields need to be transformed and pushed to the git server for compilation

In [16]:
# grab dataset 
dev = pd.read_excel(git_repo_loc + "/AidDatasGlobalChineseDevelopmentFinanceDataset_v2.0.xlsx", sheet_name='Global_CDF2.0')
mil = pd.read_excel(git_repo_loc + "/AidDatasGlobalChineseDevelopmentFinanceDataset_v2.0.xlsx", sheet_name='Military')
hu = pd.read_excel(git_repo_loc + "/AidDatasGlobalChineseDevelopmentFinanceDataset_v2.0.xlsx", sheet_name='Huawei')

df = pd.concat([dev, mil, hu])

In [17]:
geoaid_data = df[~df['geoJSON URL DL'].isna()].reset_index()

# gather all of the json files from the public github
# Note: all of the jsons are also available on the private LAS-BRI-Git repo to support the storage of additional shapefiles
geo_base = gpd.read_file(geoaid_data['geoJSON URL DL'][0])
for i in range(1, len(geoaid_data)): 
    temp = gpd.read_file(geoaid_data['geoJSON URL DL'][i])
    geo_base = geo_base.append(temp)

print(str(geo_base.shape[0]) + " out of " + str(geoaid_data.shape[0]) + " shapefiles have been retrieved.")  

3322 out of 3325 shapefiles have been retrieved.


In [18]:
geo_base_ord = geo_base

In [19]:
# restrict columns 
geo_base = geo_base[['AidData TUFF Project ID', 'Title', 'Recipient', 'geometry']]

# match with country codes 
geo_base['Recipient'] = geo_base['Recipient'].replace(country_mapping)

# add in iso codes for the countries 

geo_base = geo_base.merge(arcgis_countries[['country', 'iso']], right_on = 'country', left_on = 'Recipient', how='left')
geo_base = geo_base[['country', 'iso', 'AidData TUFF Project ID', 'Title', 'geometry']]


In [20]:
# make sure that all areas have one unique recipient 
# utilize the logged shapefile to determine the country 

countryless = geo_base[geo_base['country'].isna()]
for i in range(0, len(countryless)): 
    center = countryless['geometry'][countryless.index[i]].centroid
    g = rg.search((center.x, center.y))
    country = g[0]['cc']
    geo_base['country'][countryless.index[i]] = country

Loading formatted geocoded file...


In [21]:
# some of these shapes are unnessecary. We are better off putting them as lat/long coordinates. 
# if the area is only one group and small enough (smaller than .1), restrict and make coordinates 
# geo_base['area'] = geo_base['geometry'].area
# cord = geo_base[geo_base['area'] < .1].reset_index()
# reg = geo_base[geo_base['area'] >= .1]

__For coordinate data__
- replace with center of multipolygon 
- ensure that the point doesn't match in the existing database of points 
- add the point to the coordinate data instead

In [22]:
# if the shape is small enough, replace with a point marker 
geo_base['area'] = geo_base['geometry'].area
cord = geo_base[geo_base['area'] < .1].reset_index().drop(columns=['index'])
cord['geometry'] = cord['geometry'].centroid

In [23]:
# determine if any of these coordinates are already another city 
# if they are, use the feild temp_id to point these projects to that city_id

error = .005
count = 0 
cord['city_temp_id'] = "0"
for i in range(0, len(cord)): 
    a = cord['geometry'][i].x
    b = cord['geometry'][i].y
    
    sample = geo_cities.loc[(geo_cities['geometry'].x > a - error) & (geo_cities['geometry'].x < a + error) &
                   (geo_cities['geometry'].y > b - error) & (geo_cities['geometry'].y < b + error)].reset_index()
    if len(sample) >= 1: 
        print("found " + sample['name'][0])
        count = count + 1 
        cord['city_temp_id'][i] = str(sample['id'][0])
    
print("At error " + str(error) + " we found, " + str((count / len(cord)) * 100) + "% entities duplicated")

found Santiago
found Uvira
found Banjul
found Banjul
found Saint George’s
found Kisumu
found Arafat
found Kolonia
found Nuku‘alofa
found Nuku‘alofa
found Kyiv
found Tirana
found Mbanza Kongo
found Lucapa
found Gaborone
found Praia
found Roseau
found Ulaanbaatar
found Dhaka
found Gaborone
found Avarua
found Uvira
found Uvira
found Saint George’s
found Conakry
found Kingston
found Male
found San Fernando
found Dhaka
found Praia
found Tadjourah
found Roseau
found Roseau
found San Fernando
found Bridgetown
found Praia
found Medellín
found Roseau
found Roseau
found Kingston
found Pljevlja
found Jaffna
found Norak
found Samarkand
found Saurimo
found Mbanza Kongo
found Praia
found Praia
found Abidjan
found Manpo
found Djibouti
found Djibouti
found Djibouti
found Roseau
found Maralal
found Nuku‘alofa
found Puerto La Cruz
found Saint John’s
found Veliko Tarnovo
found Praia
found Praia
found Djibouti
found Nairobi
found Moulay Driss Zerhoun
found Lima
found Sanaa
found Praia
found Roseau
found R

In [24]:
# grab a nearby city name for the projects left 
cord['name_nearby'] = ""
for i in range(0, len(cord)):
    if cord['city_temp_id'][i] == "0": 
        name = find_nearby_city(cord['geometry'][i], 'city')
        cord['name_nearby'][i] = name

In [25]:
# the naming convention for the 'city' will be an "ID number, near city_name, country" 
cord['name'] = [str(cord['AidData TUFF Project ID'][i]) + ", near " + cord['name_nearby'][i] for i in range(0, len(cord))]

# now we need to store each new location in the geo_cities 
# we need to do it one by one so that new locations will be considered when generating the next key 
for i in range(0, len(cord)): 
    if cord['city_temp_id'][i] == "0": 
        # calculate next id 
        next_id = add_next_loc(cord['country'][i], geo_cities) 
        # adds the id to our city ids and then adds the city to the main listing
        cord['city_temp_id'][i] = next_id
        geo_cities = add_loc(geo_cities, cord['name'][i], next_id, cord['country'][i], cord['geometry'][i])

This country ZA was not found
This country IT was not found
This country IT was not found
This country IT was not found
This country MA was not found
This country MA was not found
This country MA was not found
This country IT was not found


In [26]:
# confirming geo_cities has been appropriately updated 
geo_cities

,name,id,country,geometry,population
0,Asadābād,1921,Afghanistan,POINT (71.15280 34.87420),48400.0
1,Aībak,1922,Afghanistan,POINT (68.03940 36.25340),4938.0
2,Baghlān,1923,Afghanistan,POINT (68.70000 36.13280),73031.0
3,Balkh,1924,Afghanistan,POINT (66.89890 36.75810),77000.0
4,Barakī Barak,1925,Afghanistan,POINT (68.96670 33.96670),22305.0
...,...,...,...,...,...
44147,"2089.0, near Pale, GQ",07468,Ghana,POINT (-0.99988 5.97567),NaN
44148,"31787.0, near Kita, ML",096164,Angola,POINT (13.26765 -8.81946),NaN
44149,"38913.0, near Vuorea, NO",200210,Pakistan,POINT (73.15595 33.64985),NaN
44150,"35702.0, near Vuorea, NO",200211,Pakistan,POINT (74.35516 31.57963),NaN


__For shape regional data__
- these are large enough shapes that we want to track them in their entirity 
- ensure that the shape doesn't match in the existing database of regions

In [27]:
# create the regional dataframe with the appropriate information 
regions = gpd.GeoDataFrame(columns=['name', 'id', 'country', 'geometry'], geometry='geometry')
regions

,name,id,country,geometry


In [28]:
# define the regions to keep as region that GeoAid data has as custom shapefiles 
reg = geo_base[geo_base['area'] >= .1].reset_index().drop(columns=["index"])
reg['center'] = reg['geometry'].centroid

- identify the region the custom region is nearest 
- create ids for the regions using the same logic as cities 
- name = TUFF id, near region, country 

In [29]:
# grab a nearby region name for the projects 
reg['name_nearby'] = ""
for i in range(0, len(reg)):
    name = find_nearby_city(reg['center'][i], 'region')
    reg['name_nearby'][i] = name

In [30]:
# start building the regional codes 
# the naming convention for the 'region' will be an "ID number, near region_name, country" 
reg['name'] = [str(reg['AidData TUFF Project ID'][i]) + ", near " + reg['name_nearby'][i] for i in range(0, len(reg))]
reg['region_temp_id'] = "0"

# now we need to store each new location in the regional geoframe 
# we need to do it one by one so that new locations will be considered when generating the next key 
for i in range(0, len(reg)):  
    # calculate next id 
    next_id = add_next_loc(reg['country'][i], regions) 
    # adds the id to our city ids and then adds the city to the main listing
    reg['region_temp_id'][i] = next_id
    regions = add_loc(regions, reg['name'][i], next_id, reg['country'][i], reg['geometry'][i])

### We have three unique dataframes to export 

This includes:   

| level | name | dataframetoexport | shape |    
|---------|---------|--------|----------|      
| level 1 | countries | arcgis_countries | multipolygons |       
| level 2 | regions | regions | multipolygons|       
| level 3 | cities | geo_cities | point |       

This data is being stored on S3.
Files will be manually updated. 

Store somewhere locally. 

In [31]:
default_path = os.path.expanduser("~/Desktop")

In [72]:
arcgis_countries.to_csv(default_path + "/countries.csv", index=False)
regions.to_csv(default_path + "/regions.csv", index=False)

geo_cities['latitude'] = geo_cities['geometry'].y
geo_cities['longitude'] = geo_cities['geometry'].x
geo_cities.drop(columns=['geometry']).to_csv(default_path + "/cities.csv", index=False)

### We also need to update GeoAid Data with the id's of our reg / cord data
so that we can find them later when processing that dataset

In [33]:
# check and ensure that we have a mapping of all geoaid data entities to a location
# this dataframe will be exported as ['AidData TUFF Project ID', 'location_id']

v2_geocoded = pd.DataFrame(columns=['AidData TUFF Project ID', 'city_temp_id', 'region_temp_id'])
v2_geocoded = v2_geocoded.append(cord[['AidData TUFF Project ID', 'city_temp_id']])
v2_geocoded = v2_geocoded.append(reg[['AidData TUFF Project ID', 'region_temp_id']])

In [34]:
if len(v2_geocoded) == (len(reg) + len(cord)): 
    print("all entities accounted for")

all entities accounted for


In [94]:
v2_geocoded['country_id'] = [x if str(x) != 'nan' else y for x, y in zip(v2_geocoded['region_temp_id'].str[:3], v2_geocoded['city_temp_id'].str[:3])]
v2_geocoded['city_temp_id'] = v2_geocoded['city_temp_id'].str[3:]
v2_geocoded['region_temp_id'] = v2_geocoded['region_temp_id'].str[3:]

# export the mapping 
v2_geocoded.to_csv(git_repo_loc + "/geoaid_data_to_loc_mapping.csv")